In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             

sns.set_style("darkgrid")

PATH = '/home/robot/Anand/'
sys.path.append(PATH)
print(sys.path)

from NeuralBlocks.models import LeNet
from NeuralBlocks.trainers import SupervisedTrainer

['/home/robot/miniconda3/envs/main/lib/python36.zip', '/home/robot/miniconda3/envs/main/lib/python3.6', '/home/robot/miniconda3/envs/main/lib/python3.6/lib-dynload', '', '/home/robot/miniconda3/envs/main/lib/python3.6/site-packages', '/home/robot/miniconda3/envs/main/lib/python3.6/site-packages/IPython/extensions', '/home/robot/.ipython', '/home/robot/Anand/']


In [2]:
torch.manual_seed(2456)
# cudnn.deterministic = True
cudnn.benchmark = True #False
np.random.seed(2456)

NUM_EPOCH = 200
BATCH_SIZE = 128
CHECKPOINT_INTERVAL = 100
LRS = [0.0001, 0.001, 0.01]
NORMS =[None,'BN', 'SN', 'WN','MWN', 'MSN', 'MSNTReLU', 'MWNTReLU']
DATA_PATH = PATH+"NeuralBlocks/data_utils/datasets/MNIST/"
SAVE_PATH = PATH+"NeuralBlocks/experiments/MNIST/"

os.makedirs(SAVE_PATH, exist_ok=True)

In [3]:
transform_train = transforms.Compose(
    [#transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.95, 1.05)), 
     transforms.ToTensor(),
     transforms.Normalize((0.19036,), (0.34743,)),
    ])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.19036,), (0.34743,)),
    ])

trainset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

testset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [4]:
net =  LeNet(1, num_class=10, norm='BN')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
trainer = SupervisedTrainer(net, data_bunch=[trainloader, testloader], optimizer=optimizer,
                           loss_function=criterion, metrics=None, use_cuda=False)

Warning: CUDA is available on this machine. Set use_cuda = True for faster computation.

In [9]:
trainer.run(10)

Epoch,Train Loss,Test Loss,Train Accuracy,Test Accuracy
1,0.0005,0.0269,99.9917,99.31
2,0.0004,0.0275,100.0,99.31
3,0.0002,0.0254,100.0,99.34
4,0.0001,0.0272,100.0,99.3
5,0.0001,0.0275,100.0,99.26
6,0.0001,0.0271,100.0,99.31
7,0.0001,0.0277,100.0,99.24
8,0.0001,0.0284,100.0,99.29
9,0.0001,0.0289,100.0,99.27
10,0.0,0.0287,100.0,99.3


([2.3325345516204834,
  2.2347899675369263,
  2.142349680264791,
  2.059099644422531,
  1.9903801441192628,
  1.9367624521255493,
  1.8855651106153215,
  1.83096581697464,
  1.7851924631330702,
  1.7411731600761413,
  1.6857724839990789,
  1.636205404996872,
  1.5952667548106267,
  1.5571282505989075,
  1.517456297079722,
  1.4784211181104183,
  1.4473539064912235,
  1.4178419278727636,
  1.3859409627161527,
  1.3573376268148423,
  1.330820742107573,
  1.3026448786258698,
  1.2753234039182249,
  1.2525711705287297,
  1.2262007021903991,
  1.2061384389033685,
  1.1828401199093572,
  1.1618433105094093,
  1.1406131304543594,
  1.1168714781602225,
  1.0991615403083064,
  1.0829476285725832,
  1.0645663729219725,
  1.0479886102325775,
  1.0317295236246926,
  1.0142451317773924,
  0.9992301633229127,
  0.9855747583665346,
  0.9712801392261798,
  0.9557770401239395,
  0.940922761835703,
  0.9279713055917195,
  0.914806196162867,
  0.9026192372495477,
  0.8890303942892287,
  0.878631346899530

In [1]:
trainer

NameError: name 'trainer' is not defined

In [13]:
trainer.log.logger

,Epoch,Train_Loss,Test_Loss,Train_Accuracy,Test_Accuracy


In [ ]:
best_acc = 0 
for lr in tqdm_notebook(LRS):
    for norm in tqdm_notebook(NORMS):
        net =  LeNet(1, num_class=10, norm=norm).cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=lr)

#         train_loss_log =[]
#         train_acc_log = []
#         test_loss_log =[]
#         test_acc_log =[]

#         for epoch in tqdm_notebook(range(NUM_EPOCH)):
#             train(epoch)
#             test(epoch)

        np.save(SAVE_PATH+"LeNet_Train_loss_{}_{}.npy".format(norm,lr), train_loss_log)  
        np.save(SAVE_PATH+"LeNet_Test_loss_{}_{}.npy".format(norm,lr), test_loss_log)    
        np.save(SAVE_PATH+"LeNet_Train_Acc_{}_{}.npy".format(norm,lr), train_acc_log)    
        np.save(SAVE_PATH+"LeNet_Test_Acc_{}_{}.npy".format(norm,lr), test_acc_log)   
        del net

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook([None,'BN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"LeNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"LeNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"LeNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"LeNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Act_Norm_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['WN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"LeNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"LeNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"LeNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"LeNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['MSNTReLU', 'WNTReLU'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"LeNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"LeNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"LeNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"LeNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_act_Results.pdf', dpi=400)

In [ ]:
import pandas as pd
from IPython.display import display
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [9]:
import pandas as pd
columns = ['Epoch',  'Train_Loss', 'Test_Loss',
                     'Train_Accuracy', 'Test_Accuracy']
logger = pd.DataFrame(
    columns=columns)

In [10]:
logger

,Epoch,Train_Loss,Test_Loss,Train_Accuracy,Test_Accuracy


In [11]:
d = {5:[1,1,1,1,1]}

In [12]:
d

{5: [1, 1, 1, 1, 1]}

In [13]:
k = pd.DataFrame.from_dict(d)

In [15]:
logger.append(k)

,5,Epoch,Test_Accuracy,Test_Loss,Train_Accuracy,Train_Loss
0,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN
